<a href="https://colab.research.google.com/github/sreent/data-management-intro/blob/main/MongoDB%20Hand-On%20Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to MongoDB

MongoDB is a free and open-source database program, developed by [MongoDB Inc](https://www.mongodb.com/).
Latest stable release is the 4.0.0  ( 21 June 2018 ), [here](https://www.mongodb.com/download-center#community) download the MongoDB community server.

# Installing MongoDB in Google Colab

Now, we are going to install MongoDB server in Google Colab with educative purpose.<br>
Be aware that **all data will be deleted after 12 hours**.

If you need a persistent MongoDB server, you need to install it locally, download MongoDB community server (free) from [here](http://www.google.com/search?q=mongodb+download+community+server&btnI)


In [1]:
!sudo wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

# Import the public key used by the package management system
!wget -qO - https://www.mongodb.org/static/pgp/server-4.4.asc | apt-key add -

# Create a list file for MongoDB
!echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.4 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-4.4.list

# Reload the local package database
!apt-get update

# Install the MongoDB packages
!apt-get install -y mongodb-org

# Start MongoDB
!service mongod start

# Install pymongo
!pip install pymongo

# Create Data Folder
!mkdir -p /data/db

# Start MongoDB
!mongod --fork --logpath /var/log/mongodb.log --dbpath /data/db

--2024-01-28 17:36:47--  http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
Resolving archive.ubuntu.com (archive.ubuntu.com)... 185.125.190.36, 91.189.91.82, 91.189.91.83, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|185.125.190.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1318204 (1.3M) [application/x-debian-package]
Saving to: ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’

libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  1.47MB/s    in 0.9s    

2024-01-28 17:36:48 (1.47 MB/s) - ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’ saved [1318204/1318204]

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 121671 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like pro

In [2]:
!mkdir -p /data/db

In [3]:
!mongod --fork --logpath /var/log/mongodb.log --dbpath /data/db

about to fork child process, waiting until server is ready for connections.
forked process: 2655
child process started successfully, parent exiting


In [7]:
from pymongo import MongoClient
try:
    client = MongoClient('localhost', 27017)
    print("MongoDB is running")
except Exception as e:
    print("Error connecting to MongoDB: ", e)


MongoDB is running


In [6]:
!sudo apt-get install netstat

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package netstat


In [17]:
!service mongod start

mongod: unrecognized service


In [24]:
!sudo apt-get install -y mongodb-org


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 mongodb-org-mongos : Depends: libssl1.1 (>= 1.1.1) but it is not installable
 mongodb-org-server : Depends: libssl1.1 (>= 1.1.1) but it is not installable
E: Unable to correct problems, you have held broken packages.


In [34]:
!cat /etc/mongod.conf

# mongod.conf

# for documentation of all options, see:
#   http://docs.mongodb.org/manual/reference/configuration-options/

# Where and how to store data.
storage:
  dbPath: /var/lib/mongodb
#  engine:
#  wiredTiger:

# where to write logging data.
systemLog:
  destination: file
  logAppend: true
  path: /var/log/mongodb/mongod.log

# network interfaces
net:
  port: 27017
  bindIp: 127.0.0.1


# how the process runs
processManagement:
  timeZoneInfo: /usr/share/zoneinfo

#security:

#operationProfiling:

#replication:

#sharding:

## Enterprise-Only Options:

#auditLog:

#snmp:


In [35]:
!ls /etc/init.d

dbus  fio  hwclock.sh  kmod  procps  rsync  x11-common


In [20]:
%%writefile /etc/init.d/mongod
#! /bin/sh
### BEGIN INIT INFO
# Provides:          mongodb
# Default-Start:     2 3 4 5
# Default-Stop:      0 1 6
# Short-Description: Mongodb startup script
# Description:       Mongodb start stop daemon sends SIGINT to terminate
#                    say man signal to see details

# Please check the startup params and replication options
#Mongo db usage:
#  run                      run db
#  msg [msg] [port]         send a request to the db server listening on port (or default)
#  msglots                  send many test messages, and then wait for answer on the last one
#  longmsg                  send a long test message to the db server
#  quicktest                just check basic assertions and exit
#  test2                    run test2() - see code
#
#Options:
# --help                    show this usage information
# --port <portno>           specify port number, default is 27017
# --dbpath <root>           directory for datafiles, default is /data/db/
# --quiet                   quieter output
# --cpu                     show cpu+iowait utilization periodically
# --noauth                  run without security
# --auth                    run with security
# --verbose
# -v+                       increase verbose level -v = --verbose
# --objcheck                inspect client data for validity on receipt
# --quota                   enable db quota management
# --appsrvpath <path>       root directory for the babble app server
# --nocursors               diagnostic/debugging option
# --nohints                 ignore query hints
# --nohttpinterface         disable http interface
# --nojni
# --oplog<n>                0=off 1=W 2=R 3=both 7=W+some reads
# --oplogSize <size_in_MB>  custom size if creating new replication operation log
# --sysinfo                 print some diagnostic system information
#
#Replication:
# --master
# --slave
# --source <server:port>    when a slave, specifies master
# --only <dbname>           when a slave, only replicate db <dbname>
# --pairwith <server:port> <arbiter>
# --autoresync

### END INIT INFO

# Author: Kunthar <kunthar@gmail.com> (with very slight edits by colin@mixedcontent.com)
#

# Do NOT "set -e"

# Check the paths and data dir with additional options at startup...
PATH=/sbin:/usr/bin/mongod
DESC="Mongodb Service"
NAME=mongod
DAEMON=/usr/bin/mongod
DBPATH=/data/db
#If security matters, please change arg to --auth
DAEMON_ARGS="--dbpath $DBPATH --noauth"
PIDFILE=/var/run/$NAME.pid
SCRIPTNAME=/etc/init.d/$NAME
LOGFILE=/var/log/mongod.log

# Exit if the package is not installed
[ -x "$DAEMON" ] || exit 0

# Load the VERBOSE setting and other rcS variables
. /lib/init/vars.sh

# Define LSB log_* functions.
# Depend on lsb-base (>= 3.0-6) to ensure that this file is present.
. /lib/lsb/init-functions

#
# Function that starts the daemon/service
#
do_start()
{
	echo -e "Starting $DESC \n"
        start-stop-daemon -Sm -p $PIDFILE --exec $DAEMON -- $DAEMON_ARGS >> $LOGFILE&
	echo -e "\nstarted"

}

#
# Function that stops the daemon/service
#
do_stop()
{
	echo -e "Stopping $DESC by sending ctrl+c \n"
	#Be nice send ctrl+c to mongod daemon
	start-stop-daemon --stop --signal 2 -q --pidfile $PIDFILE --name $NAME
	# Many daemons don't delete their pidfiles when they exit.
	rm -f $PIDFILE
	echo -e "\nstopped"
}

#
# Function that sends a SIGHUP to the daemon/service
#
do_reload() {
	#
	# If the daemon can reload its configuration without
	# restarting (for example, when it is sent a SIGHUP),
	# then implement that here.
	#
	start-stop-daemon --stop --signal 1 --quiet --pidfile $PIDFILE --name $NAME
	return 0
}

case "$1" in
  start)
	[ "$VERBOSE" != no ] && log_daemon_msg "Starting $DESC" "$NAME"
	do_start
	case "$?" in
		0|1) [ "$VERBOSE" != no ] && log_end_msg 0 ;;
		2) [ "$VERBOSE" != no ] && log_end_msg 1 ;;
	esac
	;;
  stop)
	[ "$VERBOSE" != no ] && log_daemon_msg "Stopping $DESC" "$NAME"
	do_stop
	case "$?" in
		0|1) [ "$VERBOSE" != no ] && log_end_msg 0 ;;
		2) [ "$VERBOSE" != no ] && log_end_msg 1 ;;
	esac
	;;
  #reload|force-reload)
	#
	# If do_reload() is not implemented then leave this commented out
	# and leave 'force-reload' as an alias for 'restart'.
	#
	#log_daemon_msg "Reloading $DESC" "$NAME"
	#do_reload
	#log_end_msg $?
	#;;
  restart|force-reload)
	#
	# If the "reload" option is implemented then remove the
	# 'force-reload' alias
	#
	log_daemon_msg "Restarting $DESC" "$NAME"
	do_stop
	case "$?" in
	  0|1)
		do_start
		case "$?" in
			0) log_end_msg 0 ;;
			1) log_end_msg 1 ;; # Old process is still running
			*) log_end_msg 1 ;; # Failed to start
		esac
		;;
	  *)
	  	# Failed to stop
		log_end_msg 1
		;;
	esac
	;;
  *)
	#echo "Usage: $SCRIPTNAME {start|stop|restart|reload|force-reload}" >&2
	echo "Usage: $SCRIPTNAME {start|stop}" >&2
	exit 3
	;;
esac

:

Overwriting /etc/init.d/mongod


In [23]:
!chmod 755 /etc/init.d/mongod
!chown root:root /etc/init.d/mongod

In [25]:
!sudo /etc/init.d/mongod restart > /dev/null

/etc/init.d/mongod: 28: /lib/init/vars.sh: cat: not found
/etc/init.d/mongod: 97: rm: not found


In [53]:
!ulimit -n

1048576


In [66]:
!mkdir -p /data                                          # data folder creation
!mkdir -p /data/db


In [28]:
!mongosh

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	65b688ad0881e767bc8b41e8
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.1
MongoNetworkError: connect ECONNREFUSED 127.0.0.1:27017


In [15]:
!ps -ax | grep mongo

   8022 ?        S      0:00 /bin/bash -c ps -ax | grep mongo
   8024 ?        S      0:00 grep mongo


In [30]:
!mongod --quiet --eval  "printjson(db.adminCommand('listDatabases'))"

Error parsing command line: unrecognised option '--eval'
try 'mongod --help' for more information
